Ref: https://github.com/SchindlerLiang/VAE-for-Anomaly-Detection

In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [5]:
def GRU_VAE(X_train):
    latent_dim = 16

    encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = layers.GRU(64, return_sequences=True)(encoder_inputs)
    x = layers.GRU(32)(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
    x = layers.GRU(32, return_sequences=True)(x)
    x = layers.GRU(64)(x)
    x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
    decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    model = VAE(encoder, decoder)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
    history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
    return model

### Yahoo S5

In [6]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [7]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.9924811515178951 0.9846206216760173 0.9999488105959181
yahoo_A1 0.9599999424000027 0.9027291866049527 0.999530507820946
yahoo_A1 0.9565216849716472 0.9294027075970502 0.9776921087106546
yahoo_A1 0.999999930000003 0.7999999503333362 0.9999999797237571
yahoo_A1 0.7692307076923109 0.6237454881308916 0.9878968087906828
yahoo_A1 0.5454544892562023 0.31860867089890993 0.9281745880599177
yahoo_A1 0.999999948484851 0.9848484768271686 0.999999998160173
yahoo_A1 0.9714285206763874 0.9778746173646744 0.9866109727157787
yahoo_A1 0.7763712574053335 0.8605467734597745 0.9078921450580624
yahoo_A1 0.9999999487179513 0.9871794802018831 0.9999999983719279
yahoo_A1 0.6467661244028643 0.6821671943259702 0.6944332256814504
yahoo_A1 0.9999999487654349 0.9876543142436762 0.9999999984095603
yahoo_A1 0.9999999483870992 0.9838709593577399 0.9999999980537634
yahoo_A1 0.09473683303047171 0.00144927606610165 -0.0
yahoo_A1 0.34146338616624533 0.03476337050829032 0.0891075263906907
yahoo_A1 0.999999947619

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.25396823169564314 0.0013440869727441144 0.013297871007101739
yahoo_A3 0.442477841491114 0.1379907555093967 0.3141621673049646
yahoo_A3 0.47750861400366645 0.08472034602769161 0.2095632137141159
yahoo_A3 0.25396823169564314 0.0013262609306003033 -0.0
yahoo_A3 0.6314151976538852 0.16010397552740746 0.1878220358906577
yahoo_A3 0.41726615387920135 0.02950996360713502 0.05390614721571588
yahoo_A3 0.5852089616298454 0.1751140472948915 0.21343771648294085
yahoo_A3 0.45614031550631234 0.07857079497833565 0.20629850310886444
yahoo_A3 0.5210083646578659 0.31196300292156764 0.5603471636275839
yahoo_A3 0.7134502462945892 0.28918333409167984 0.32067438518250263
yahoo_A3 0.637770854195864 0.20067344277560226 0.31605807536732144
yahoo_A3 0.5783521396394755 0.2956132128764962 0.3922187462587988
yahoo_A3 0.19301846297450495 0.043216702119184915 0.21687562217144915
yahoo_A3 0.6766916843462069 0.17366922145504674 0.0937628408882275
yahoo_A3 0.11563168070466742 0.0012077301518444415 -0.0
yahoo_

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.4853700147102329 0.25657498107801496 0.49332625438119043
yahoo_A4 0.25396823169564314 0.044770499351400754 0.21725193406830898
yahoo_A4 0.41726615387920135 0.055755066218204294 0.11679016066829237
yahoo_A4 0.2945736181779962 0.05265633962775768 0.1493759594894746
yahoo_A4 0.6850828225878368 0.7806503313212262 0.7872918848021806
yahoo_A4 0.7349822816366827 0.8264313639880828 0.8701555013750565
yahoo_A4 0.43694490352936993 0.3341806262531949 0.5732274134055915
yahoo_A4 0.3673469086517006 0.010607232062204992 0.10479702453898934
yahoo_A4 0.5942491593197877 0.16722972163773106 0.17894611908898933
yahoo_A4 0.45070419027970904 0.21445981427106683 0.3850530805342578
yahoo_A4 0.637770854195864 0.031129949794456806 0.06507821794702008
yahoo_A4 0.3882783568463308 0.10804331585128504 0.4014503416980334
yahoo_A4 0.45070419027970904 0.02380148439014991 0.1616334593335847
yahoo_A4 0.3551401575753364 0.10951125947592956 0.3103227409262222
yahoo_A4 0.07439824225732532 0.0011792458868547162 

In [8]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.721876,0.611384,0.741474
yahoo_A2,0.548413,0.370599,0.683172
yahoo_A3,0.525552,0.253386,0.350647
yahoo_A4,0.478258,0.218550,0.333221


### NASA

In [9]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [10]:
for loader in [load_nasa]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.25300017462469454 0.1328442985983932 0.36693179617049704
D2 0.2699530282282525 0.58400092990081 0.5173913038147389


In [11]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.253000,0.132844,0.366932
D2,0.269953,0.584001,0.517391


### SMD

In [12]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_smd]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.20232181144029696 0.5556857798257031 0.521888680406877
smd 0.6787003111164003 0.5490042926100768 0.9186154755807889
smd 0.25058153864288274 0.22078452988844366 0.5955408869659128
smd 0.2701578910395887 0.29703188922447676 0.5964809416824509
smd 0.689655126355723 0.7311883672195485 0.9524332230177086


In [ ]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

### ECG

In [ ]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_ecg]:
    datasets = loader(32, 16)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

In [ ]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

### Power Demand

In [ ]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_power_demand]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

In [ ]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

### 2D Gesture

In [ ]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_gesture]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        model = GRU_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

In [ ]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()